# I . Strategy Pattern 
### _In Python_¨

## I.1 The OOP implementation

The Strategy Pattern defines a **family of algorithms**, encapsulates each one of them and makes them interchangeable. This pattern favours the modificacion of these algorithms without affecting the clients using it.

- The encapsulation is what makes the algorithms interchangeable.
- i.e., this pattern de-couples the algorithms from the object (client) using them. So, changing the algorithms does not affect the client's code


Why use it?

1. Inheritance is not intended for code reuse. i.e., strategy pattern minimizes inheritance.
2. The strategy pattern avoids the hard coding of behaviours into the intended overall classes. It favours the creation of abstract behaviours (by encapsulating each algorithm in its own class), which are latter injected into the class using them.

#### Example: List classifier
*(This is the most 'straight forward implementation from the books in Python, using the abc library to create the abstract classes)*

In [5]:
from abc import ABC, abstractmethod
import random

#We create the abstract class for the classification method
class I_NumClassifier(ABC):
    @abstractmethod
    def classify(self,*args,**kwargs):
        pass

#We encapsulate an specific behaviour (algorithm or concrete behaviour) in its own class
class RandomClassifier(I_NumClassifier):
    def classify(self,lista:list) -> list:
        lista2 = lista.copy()
        random.shuffle(lista2)
        return lista2

#We create the client that uses the classification algorithm
class List_reader:
    def __init__(self,lista:list) -> None:
        self.lista = lista
    
    def classify_list(self,strategy: I_NumClassifier):
        print(f'The provided classifier is {strategy().__class__.__name__}')
        print(f'Given list {self.lista}')
        sorted_list = strategy().classify(self.lista)
        print(f'Classified list {sorted_list}')

Let us try some examples now

In [7]:
lista = [1,4,7,0,12,5,43]
reader = List_reader(lista)
reader.classify_list(RandomClassifier)


The provided classifier is RandomClassifier
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list [0, 7, 12, 4, 1, 43, 5]


We can always create other concrete behaviours and inject them into the client, without changing the client's code

In [8]:
class ReversedClassifier(I_NumClassifier):
    def classify(self,lista:list) -> list:
        lista2 = lista.copy()
        lista2.reverse()
        return lista2

class SortingClassifier(I_NumClassifier):
    def classify(self,lista:list) -> list:
        lista2 = lista.copy()
        lista2.sort()
        return lista2

class BlackHoleClassifier(I_NumClassifier):
    def classify(self,lista:list) -> list:
        return []

Let us try them

In [9]:
reader.classify_list(ReversedClassifier)
reader.classify_list(SortingClassifier)
reader.classify_list(BlackHoleClassifier)



The provided classifier is ReversedClassifier
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list [43, 5, 12, 0, 7, 4, 1]
The provided classifier is SortingClassifier
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list [0, 1, 4, 5, 7, 12, 43]
The provided classifier is BlackHoleClassifier
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list []


Notice how we didn't modify the List_reader class at all, and we didn't even required a re-instantiation of the class. It works perfectly well by calling different strategies from the execute() method

## Functional Approach to the Strategy Pattern

Given that Python provides a good basis for functional programming, let usw try this option.
Instead of creating classes for the concrete strategies, we can create functions that are later directly injected into the client

In [4]:
def reversedClassifier_function(lista : list) -> list:
    lista2 = lista.copy()
    lista2.reverse()
    return lista2

def blackHoleClassifier_function(lista : list) -> list:
    return []

def randomClassifier_function(lista : list) -> list:
    lista2 = lista.copy()
    random.shuffle(lista2)
    return lista2

def sortingClassifier_function(lista:list) -> list:
    lista2 = lista.copy()
    lista2.sort()
    return lista2


#We also have to modify the client to accept functions

class List_reader_functional:
    def __init__(self,lista:list) -> None:
        self.lista = lista
    
    def classify_list(self,strategy):
        print(f'The provided classifier is {strategy.__name__}')
        print(f'Given list {self.lista}')
        sorted_list = strategy(self.lista)
        print(f'Classified list {sorted_list}')




Let us try again all the different combinations of strategies

In [7]:
lista = [1,4,7,0,12,5,43]
reader = List_reader_functional(lista)
reader.classify_list(randomClassifier_function)
reader.classify_list(reversedClassifier_function)
reader.classify_list(sortingClassifier_function)
reader.classify_list(blackHoleClassifier_function)

The provided classifier is randomClassifier_function
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list [0, 7, 12, 5, 43, 4, 1]
The provided classifier is reversedClassifier_function
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list [43, 5, 12, 0, 7, 4, 1]
The provided classifier is sortingClassifier_function
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list [0, 1, 4, 5, 7, 12, 43]
The provided classifier is blackHoleClassifier_function
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list []


And it works just fine! (Still, a strategy pattern)